In [ ]:
!pip install d2l

In [ ]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

生成自定义样本

In [ ]:
def synthetic_data(w, b, num_examples):
    """生成y=Xw+b+噪声"""
    #生成样本矩阵 均值和方差分别为0，1  样本数为num_examples，特征数为len(w)
    X = torch.normal(0, 1, (num_examples, len(w)))
    #得到标签向量y
    y = torch.matmul(X, w) + b
    #给标签向量y加噪声，噪声跟y的shape相同，均值0方差0.01
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [ ]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.1807, -1.2172]) 
label: tensor([7.9818])


定义一个data_iter函数， 该函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为batch_size的小批量。

In [ ]:
def data_iter(batch_size, features, labels):
    # 获取样本数量
    num_examples = len(features)
    # 按样本创建下标
    indices = list(range(num_examples))
    # 打乱下标
    random.shuffle(indices)
    # 每次循环取出batch_size个样本和对应标签出来
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

试试上面这个分批函数效果

In [ ]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break


tensor([[ 0.5683, -1.2830],
        [-1.9906,  0.0136],
        [ 0.1531, -0.7104],
        [ 0.5081,  2.4008],
        [ 0.2598, -1.6010],
        [ 0.6648,  0.1008],
        [ 2.7117,  1.0925],
        [ 1.8315, -1.1728],
        [ 0.8145, -1.7820],
        [ 1.2800,  1.1984]]) 
 tensor([[ 9.6977],
        [ 0.1583],
        [ 6.9228],
        [-2.9486],
        [10.1637],
        [ 5.1844],
        [ 5.9143],
        [11.8539],
        [11.8786],
        [ 2.6611]])


定义 初始化模型参数

In [ ]:
# 维度为2行1列，数值取自均值为0，标准差为0.01的正态分布
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
# 偏差b取了0，标量
b = torch.zeros(1, requires_grad=True)

定义模型

In [ ]:
def linreg(X, w, b):
    """线性回归模型"""
    return torch.matmul(X, w) + b

定义损失函数

In [ ]:
def squared_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

定义优化算法

In [ ]:
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

训练

In [ ]:
lr = 0.03
num_epochs = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.000058
epoch 2, loss 0.000050
epoch 3, loss 0.000050
epoch 4, loss 0.000050
epoch 5, loss 0.000049
epoch 6, loss 0.000049
epoch 7, loss 0.000049
epoch 8, loss 0.000050
epoch 9, loss 0.000050
epoch 10, loss 0.000049






#线性回归的简洁实现

##生成数据集

In [ ]:
!pip install d2l

In [ ]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

##读取数据集

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

##定义模型

In [ ]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

##初始化模型参数

In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

##定义损失函数

In [ ]:
loss = nn.MSELoss()

##定义优化算法

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

##训练

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000257
epoch 2, loss 0.000094
epoch 3, loss 0.000094
